### Access and download hourly environmental datasets

This notebook demonstrates how to access hourly environmental datasets as a Pandas Dataframe and download to a CSV file.

In [ ]:
import ipywidgets as widgets
from ipywidgets import *
from arcgis.gis import GIS
from datetime import datetime, timedelta
import pandas as pd
from io import StringIO, BytesIO
import zipfile
import requests

pdi = GIS(url="https://pdi-development.scinet.usda.gov/portal")
print("ArcGIS portal set to " + pdi.url)

##### Choose a dataset

In [ ]:
dataset_widget = widgets.Dropdown(options=[('Mesonet', 'mesonet'), ('Hydro', 'hydro'), ('Streamgage', 'streamgage')],value='mesonet')
dataset_widget

##### Choose a date and hour (UTC)

In [ ]:
todays_date = datetime.today()
date_widget = widgets.DatePicker(description='Pick a Date',disabled=False,value=todays_date)
display(date_widget)
hour_widget=widgets.Dropdown(options=['0000', '0100', '0200', '0300', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300'],description='Hour (UTC):',disabled=False)
display(hour_widget)

##### Query the portal for the item

In [ ]:
item_name = f'{dataset_widget.value}-{date_widget.value.strftime("%Y%m%d")}_{hour_widget.value}'
csv_item = pdi.content.search(query=f'{item_name} type:CSV')[0]
csv_item

##### Download the item to a Dataframe

In [ ]:
csv_url = f"https://pdi-development.scinet.usda.gov/portal/sharing/rest/content/items/{csv_item.id}/data"
response = requests.get(csv_url)

if response.status_code == 200:
    if response.headers['content-type'] == 'application/zip':
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            # Assuming there's only one CSV file in the zip archive
            csv_file = [f for f in z.namelist() if f.endswith('.csv')][0]
            with z.open(csv_file) as f:
                df = pd.read_csv(f)
    else:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data)
    
    df = df.drop(['fid', 'x', 'y'], axis=1)
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = round(df[col], 2)
else:
    print(f"Failed to download file. Status code: {response.status_code}")
df

##### Convert to a CSV file

In [ ]:
df.to_csv(f'{item_name}.csv', escapechar='"', index=False)